In [71]:
import pandas as pd

data = pd.read_excel('C:/Users/daniel.oliveira/downloads/KF_data.xlsx')

data['date'] = pd.to_datetime(data['date'], format='%Y%m')
data = data.set_index('date')
data = data.astype('float')

factors = data[['VWMe', 'SMB', 'HML']].values
rf = data[['RF']].values
stocks = data.iloc[:,6:].values

excess_returns = stocks - rf

In [60]:
excess_returns

array([[-0.86, -1.64, -0.86, ...,  1.78,  2.71,  0.34],
       [ 3.56,  0.96,  4.57, ...,  1.57,  5.39,  7.51],
       [-5.42,  2.85,  0.52, ..., -0.46, -0.53, -2.66],
       ...,
       [14.95, 14.  ,  9.79, ..., 12.82,  9.16, 12.72],
       [-3.15, -0.1 , -0.57, ...,  0.84,  0.72, -4.46],
       [ 1.55,  1.5 ,  1.75, ...,  2.74,  3.2 ,  0.61]])

In [63]:
factors

array([[ 2.620e+00, -2.160e+00, -2.920e+00],
       [ 2.560e+00, -1.490e+00,  4.880e+00],
       [ 3.600e-01, -1.380e+00, -1.000e-02],
       ...,
       [ 1.153e+01,  3.530e+00, -9.000e-01],
       [-6.100e-01, -2.200e-01, -2.000e-02],
       [ 4.900e-01, -4.400e-01,  1.560e+00]])

In [33]:
import statsmodels.api as sm

# Time series regressions
x = sm.add_constant(factors)
ts_res = sm.OLS(excess_returns, x).fit()
alpha = ts_res.params[0]
beta = ts_res.params[1:]

In [39]:
import numpy as np

avgexcess_returns = np.mean(excess_returns, 0)
# Cross-section regression
cs_res = sm.OLS(avgexcess_returns.T, beta.T).fit()
risk_premia = cs_res.params

In [42]:
cs_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.918
Model:                            OLS   Adj. R-squared (uncentered):              0.905
Method:                 Least Squares   F-statistic:                              74.31
Date:                Tue, 20 Oct 2020   Prob (F-statistic):                    5.09e-11
Time:                        06:58:03   Log-Likelihood:                         -1.4916
No. Observations:                  23   AIC:                                      8.983
Df Residuals:                      20   BIC:                                      12.39
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.5479      0.092      5.951      0.000       0.356       0.740
x2             0.3872      0.122      3.175      0.005       0.133       0.642
x3             0.1907      0.147      1.298      0.209      -0.116       0.497
==============================================================================
Omnibus:                       37.924   Durbin-Watson:                   1.192
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              107.268
Skew:                          -2.948   Prob(JB):                     5.09e-24
Kurtosis:                      11.784   Cond. No.                         3.36
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [80]:
panel_excess_returns = pd.DataFrame(excess_returns)
panel_excess_returns.columns = data.iloc[:,6:].columns
#panel_excess_returns.index = pd.to_datetime(data.iloc[:,6:].index)
panel_excess_returns = panel_excess_returns.reset_index()
panel_excess_returns = panel_excess_returns.melt(['date'])
panel_excess_returns = panel_excess_returns.set_index(['date', 'variable'])
panel_excess_returns.head()

,,value
date,variable,
1926-07-01,S1V3,-0.86
1926-08-01,S1V3,3.56
1926-09-01,S1V3,-5.42
1926-10-01,S1V3,-4.12
1926-11-01,S1V3,2.00


In [81]:
panel_x = pd.DataFrame(x)
panel_x.columns = ['constant'] + ['VWMe', 'SMB', 'HML']
panel_x.index = pd.to_datetime(data.iloc[:,6:].index)
panel_x = panel_x.reset_index()
panel_x = panel_x.set_index(['date'])
panel_x.head()

,constant,VWMe,SMB,HML
date,,,,
1926-07-01,1.0,2.62,-2.16,-2.92
1926-08-01,1.0,2.56,-1.49,4.88
1926-09-01,1.0,0.36,-1.38,-0.01
1926-10-01,1.0,-3.43,0.04,0.71
1926-11-01,1.0,2.44,-0.24,-0.31


In [82]:
from linearmodels.panel import PanelOLS

mod = PanelOLS(panel_excess_returns, panel_x, entity_effects=True)
fe_res = mod.fit()
print(fe_res)

ValueError: The index on the time dimension must be either numeric or date-like